In [38]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [2]:
spark=SparkSession.builder.appName('binary_class').getOrCreate()

In [3]:
df=spark.read.csv('classification_data.csv',inferSchema = True,header=True)

In [4]:
print((df.count(),len(df.columns)))

(46751, 12)


In [5]:
df.printSchema()

root
 |-- loan_id: string (nullable = true)
 |-- loan_purpose: string (nullable = true)
 |-- is_first_loan: integer (nullable = true)
 |-- total_credit_card_limit: integer (nullable = true)
 |-- avg_percentage_credit_card_limit_used_last_year: double (nullable = true)
 |-- saving_amount: integer (nullable = true)
 |-- checking_amount: integer (nullable = true)
 |-- is_employed: integer (nullable = true)
 |-- yearly_salary: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- dependent_number: integer (nullable = true)
 |-- label: integer (nullable = true)



In [6]:
df.show(5)

+-------+------------+-------------+-----------------------+-----------------------------------------------+-------------+---------------+-----------+-------------+---+----------------+-----+
|loan_id|loan_purpose|is_first_loan|total_credit_card_limit|avg_percentage_credit_card_limit_used_last_year|saving_amount|checking_amount|is_employed|yearly_salary|age|dependent_number|label|
+-------+------------+-------------+-----------------------+-----------------------------------------------+-------------+---------------+-----------+-------------+---+----------------+-----+
|    A_1|    personal|            1|                   7900|                                            0.8|         1103|           6393|          1|        16400| 42|               4|    0|
|    A_2|    personal|            0|                   3300|                                           0.29|         2588|            832|          1|        75500| 56|               1|    0|
|    A_3|    personal|            0|    

In [7]:
df.groupBy('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|    1|16201|
|    0|30550|
+-----+-----+



In [8]:
df.groupBy('loan_purpose').count().show()

+------------+-----+
|loan_purpose|count|
+------------+-----+
|      others| 6763|
|   emergency| 7562|
|    property|11388|
|  operations|10580|
|    personal|10458|
+------------+-----+



In [9]:
loan_purpose_indexer = StringIndexer(inputCol="loan_purpose", outputCol="loan_index").fit(df)

In [10]:
df = loan_purpose_indexer.transform(df)

In [11]:
loan_encoder = OneHotEncoder(inputCol="loan_index", outputCol="loan_purpose_vec")

In [13]:
df = loan_encoder.fit(df).transform(df)

In [15]:
df.select(['loan_purpose','loan_index','loan_purpose_vec']).show(3,False)

+------------+----------+----------------+
|loan_purpose|loan_index|loan_purpose_vec|
+------------+----------+----------------+
|personal    |2.0       |(4,[2],[1.0])   |
|personal    |2.0       |(4,[2],[1.0])   |
|personal    |2.0       |(4,[2],[1.0])   |
+------------+----------+----------------+
only showing top 3 rows



In [17]:
df_assembler = VectorAssembler(inputCols=['is_first_loan',
'total_credit_card_limit',
'avg_percentage_credit_card_limit_used_last_year',
'saving_amount',
'checking_amount',
'is_employed',
'yearly_salary',
'age',
'dependent_number',
'loan_purpose_vec'], outputCol="features")

In [18]:
df = df_assembler.transform(df)

In [20]:
df.select(['features','label']).show(10,False)

+--------------------------------------------------------------------+-----+
|features                                                            |label|
+--------------------------------------------------------------------+-----+
|[1.0,7900.0,0.8,1103.0,6393.0,1.0,16400.0,42.0,4.0,0.0,0.0,1.0,0.0] |0    |
|[0.0,3300.0,0.29,2588.0,832.0,1.0,75500.0,56.0,1.0,0.0,0.0,1.0,0.0] |0    |
|[0.0,7600.0,0.9,1651.0,8868.0,1.0,59000.0,46.0,1.0,0.0,0.0,1.0,0.0] |0    |
|[1.0,3400.0,0.38,1269.0,6863.0,1.0,26000.0,55.0,8.0,0.0,0.0,1.0,0.0]|0    |
|[0.0,2600.0,0.89,1310.0,3423.0,1.0,9700.0,41.0,4.0,0.0,0.0,0.0,1.0] |1    |
|[0.0,7600.0,0.51,1040.0,2406.0,1.0,22900.0,52.0,0.0,0.0,1.0,0.0,0.0]|0    |
|[1.0,6900.0,0.82,2408.0,5556.0,1.0,34800.0,48.0,4.0,0.0,1.0,0.0,0.0]|0    |
|[0.0,5700.0,0.56,1933.0,4139.0,1.0,32500.0,64.0,2.0,0.0,0.0,1.0,0.0]|0    |
|[1.0,3400.0,0.95,3866.0,4131.0,1.0,13300.0,23.0,3.0,0.0,0.0,1.0,0.0]|0    |
|[0.0,2900.0,0.91,88.0,2725.0,1.0,21100.0,52.0,1.0,0.0,0.0,1.0,0.0]  |1    |

In [21]:
model_df=df.select(['features','label'])

In [22]:
training_df,test_df=model_df.randomSplit([0.75,0.25])

In [24]:
log_reg=LogisticRegression().fit(training_df)

In [25]:
lr_summary=log_reg.summary

In [26]:
lr_summary.accuracy

0.8947398464480406

In [27]:
lr_summary.areaUnderROC

0.9593413356398106

In [28]:
predictions = log_reg.transform(test_df)

In [29]:
predictions.show(10)

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(13,[0,1,2,3,4,7]...|    0|[5.13947703551435...|[0.99417339442305...|       0.0|
|(13,[0,1,2,3,4,7]...|    1|[1.25913230953471...|[0.77887670353559...|       0.0|
|(13,[0,1,2,3,4,7,...|    1|[-4.5547664216351...|[0.01040750124113...|       1.0|
|(13,[0,1,2,3,4,7,...|    1|[-4.1271009933436...|[0.01587353749532...|       1.0|
|(13,[0,1,2,3,4,7,...|    1|[-6.3909538031780...|[0.00167384976778...|       1.0|
|(13,[0,1,2,3,4,7,...|    1|[-6.0150520203242...|[0.00243577386109...|       1.0|
|(13,[0,1,2,3,4,7,...|    1|[-5.0853555762749...|[0.00614864710582...|       1.0|
|(13,[0,1,2,3,4,7,...|    1|[-3.5667530655072...|[0.02747142550903...|       1.0|
|(13,[0,1,2,3,4,7,...|    0|[-1.2421098491049...|[0.22406894948663...|       1.0|
|(13,[0,1,2,3,4,

In [30]:
model_predictions = log_reg.transform(test_df)

In [31]:
model_predictions = log_reg.evaluate(test_df)

In [32]:
model_predictions.accuracy

0.8919241932730066

In [34]:
rf = RandomForestClassifier()

In [35]:
rf_model = rf.fit(training_df)

In [36]:
model_predictions = rf_model.transform(test_df)

In [39]:
evaluator = BinaryClassificationEvaluator()

In [40]:
rf = RandomForestClassifier()

In [44]:
paramGrid = (ParamGridBuilder().addGrid(rf.maxDepth, [5,10,20,25,30]).addGrid(rf.maxBins, [20,30,40 ]).addGrid(rf.numTrees, [5, 20,50]).build())

In [45]:
cv = CrossValidator(estimator=rf, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

In [46]:
cv_model = cv.fit(training_df)

In [47]:
best_rf_model = cv_model.bestModel

In [49]:
model_predictions = best_rf_model.transform(test_df)

In [50]:
true_pos=model_predictions.filter(model_predictions['label']==1).filter(model_predictions['prediction']==1).count()

In [51]:
actual_pos=model_predictions.filter(model_predictions['label']==1).count()

In [52]:
pred_pos=model_predictions.filter(model_predictions['prediction']==1).count()

In [53]:
recall_rate=float(true_pos)/(actual_pos)

In [54]:
print(recall_rate)

0.9058561897702001
